In [26]:
import os, sqlite3, hashlib, time, json
import random
import pandas as pd

# ------------------- RESET DB -------------------
if os.path.exists("bankdb.sqlite"):
    os.remove("bankdb.sqlite")

db = sqlite3.connect("bankdb.sqlite")
cursor = db.cursor()

# ------------------- CREATE TABLES -------------------
cursor.execute("""
CREATE TABLE IF NOT EXISTS Users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    balance REAL DEFAULT 0.0
)""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Blocks (
    block_id INTEGER PRIMARY KEY AUTOINCREMENT,
    prev_hash TEXT,
    curr_hash TEXT,
    timestamp INTEGER
)""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Transactions (
    tx_id INTEGER PRIMARY KEY AUTOINCREMENT,
    block_id INTEGER,
    sender_id INTEGER,
    receiver_id INTEGER,
    amount REAL,
    timestamp INTEGER,
    FOREIGN KEY(block_id) REFERENCES Blocks(block_id)
)""")
db.commit()

# ------------------- BLOCKCHAIN LOGIC -------------------
def calculate_hash(prev_hash, tx_data, timestamp):
    value = str(prev_hash) + json.dumps(tx_data, sort_keys=True) + str(timestamp)
    return hashlib.sha256(value.encode()).hexdigest()

def add_transaction(block_id, sender, receiver, amount):
    cursor.execute("INSERT INTO Transactions (block_id, sender_id, receiver_id, amount, timestamp) VALUES (?, ?, ?, ?, ?)",
                   (block_id, sender, receiver, amount, int(time.time())))
    db.commit()

def create_block_with_transactions(prev_hash, transactions):
    cursor.execute("INSERT INTO Blocks (prev_hash, curr_hash, timestamp) VALUES (?, ?, ?)",
                   (prev_hash, "", int(time.time())))
    db.commit()
    block_id = cursor.lastrowid

    for tx in transactions:
        add_transaction(block_id, tx["from"], tx["to"], tx["amount"])

    cursor.execute("SELECT sender_id, receiver_id, amount FROM Transactions WHERE block_id=? ORDER BY tx_id", (block_id,))
    rows = cursor.fetchall()
    tx_data = [{"from": r[0], "to": r[1], "amount": r[2]} for r in rows]

    cursor.execute("SELECT timestamp FROM Blocks WHERE block_id=?", (block_id,))
    timestamp = int(cursor.fetchone()[0])

    curr_hash = calculate_hash(prev_hash, tx_data, timestamp)
    cursor.execute("UPDATE Blocks SET curr_hash=? WHERE block_id=?", (curr_hash, block_id))
    db.commit()

    return block_id, curr_hash

# Verify individual blocks
def verify_chain_per_block():
    cursor.execute("SELECT block_id, prev_hash, curr_hash, timestamp FROM Blocks ORDER BY block_id")
    blocks = cursor.fetchall()
    status = {}
    for i in range(len(blocks)):
        block_id, prev_hash, curr_hash, timestamp = blocks[i]
        cursor.execute("SELECT sender_id, receiver_id, amount FROM Transactions WHERE block_id=? ORDER BY tx_id", (block_id,))
        rows = cursor.fetchall()
        tx_data = [{"from": r[0], "to": r[1], "amount": r[2]} for r in rows]
        expected_hash = calculate_hash(prev_hash, tx_data, int(timestamp))
        if curr_hash != expected_hash:
            status[block_id] = "❌ Tampered"
        else:
            status[block_id] = "✅ Valid"
        if i > 0:
            prev_block = blocks[i-1]
            if prev_hash != prev_block[2]:
                status[block_id] = "❌ Chain Broken"
                status[prev_block[0]] = "❌ Chain Broken"
    return status

# ------------------- DEMO DATA -------------------
users = [("Alice","alice@mail.com",1000),
         ("Bob","bob@mail.com",500),
         ("Charlie","charlie@mail.com",200),
         ("David","david@mail.com",300)]
for u in users:
    cursor.execute("INSERT INTO Users (name, email, balance) VALUES (?, ?, ?)", u)
db.commit()

# ------------------- CREATE MULTIPLE BLOCKS -------------------
prev_hash = "0"
num_blocks = 3
blocks_info = []

print("Creating blocks...\n")
for b in range(num_blocks):
    tx_count = random.randint(2,3)
    txs = []
    for _ in range(tx_count):
        sender = random.randint(1,4)
        receiver = random.randint(1,4)
        while receiver == sender:
            receiver = random.randint(1,4)
        amount = random.randint(10,200)
        txs.append({"from": sender, "to": receiver, "amount": amount})
    block_id, curr_hash = create_block_with_transactions(prev_hash, txs)
    blocks_info.append((block_id, curr_hash))
    prev_hash = curr_hash
    print(f"✅ Block {block_id} created with {tx_count} transactions, hash: {curr_hash}")

# ------------------- VISUALIZE BLOCKCHAIN BEFORE TAMPERING -------------------
status = verify_chain_per_block()
print("\n📊 Blockchain Overview (before tampering):")

block_rows = []
for block_id, curr_hash in blocks_info:
    cursor.execute("SELECT prev_hash, timestamp FROM Blocks WHERE block_id=?", (block_id,))
    prev_hash, timestamp = cursor.fetchone()
    cursor.execute("SELECT sender_id, receiver_id, amount FROM Transactions WHERE block_id=? ORDER BY tx_id", (block_id,))
    txs = cursor.fetchall()
    tx_summary = "; ".join([f"{t[0]}→{t[1]}:${t[2]}" for t in txs])
    block_rows.append([block_id, prev_hash, curr_hash, tx_summary, status[block_id]])

df_blocks = pd.DataFrame(block_rows, columns=["Block ID", "Prev Hash", "Curr Hash", "Transactions", "Status"])

# Highlight tampered blocks in red
def highlight_tampered(s):
    color = []
    for val in s:
        if "❌" in val:
            color.append('background-color: #ff9999')  # light red
        else:
            color.append('')
    return color

display(df_blocks.style.apply(highlight_tampered, subset=['Status']))

# ------------------- TAMPERING DEMO -------------------
print("\n⚠️ Tampering with a transaction in block 1...")
cursor.execute("UPDATE Transactions SET amount = 9999 WHERE block_id = 1 LIMIT 1")
db.commit()

# ------------------- VISUALIZE BLOCKCHAIN AFTER TAMPERING -------------------
status = verify_chain_per_block()
print("\n📊 Blockchain Overview (after tampering):")

block_rows = []
for block_id, curr_hash in blocks_info:
    cursor.execute("SELECT prev_hash, timestamp FROM Blocks WHERE block_id=?", (block_id,))
    prev_hash, timestamp = cursor.fetchone()
    cursor.execute("SELECT sender_id, receiver_id, amount FROM Transactions WHERE block_id=? ORDER BY tx_id", (block_id,))
    txs = cursor.fetchall()
    tx_summary = "; ".join([f"{t[0]}→{t[1]}:${t[2]}" for t in txs])
    block_rows.append([block_id, prev_hash, curr_hash, tx_summary, status[block_id]])

df_blocks = pd.DataFrame(block_rows, columns=["Block ID", "Prev Hash", "Curr Hash", "Transactions", "Status"])
display(df_blocks.style.apply(highlight_tampered, subset=['Status']))


Creating blocks...

✅ Block 1 created with 2 transactions, hash: bf0c29f6f797ad22514562b34cf3503f4e1fadf431d378a4b7302ff774ff7ee5
✅ Block 2 created with 3 transactions, hash: 573f308ea1b53165a4820f83b661023b3a415e7e7a117dee2c134a0970c276de
✅ Block 3 created with 2 transactions, hash: 1798ecfcbfeb56ae4ce2af6649382266ea705de1f0d7b0d1dfb20ea4be81100c

📊 Blockchain Overview (before tampering):


,Block ID,Prev Hash,Curr Hash,Transactions,Status
0,1,0,bf0c29f6f797ad22514562b34cf3503f4e1fadf431d378a4b7302ff774ff7ee5,1→2:$172.0; 2→3:$75.0,✅ Valid
1,2,bf0c29f6f797ad22514562b34cf3503f4e1fadf431d378a4b7302ff774ff7ee5,573f308ea1b53165a4820f83b661023b3a415e7e7a117dee2c134a0970c276de,1→4:$109.0; 4→2:$176.0; 4→2:$91.0,✅ Valid
2,3,573f308ea1b53165a4820f83b661023b3a415e7e7a117dee2c134a0970c276de,1798ecfcbfeb56ae4ce2af6649382266ea705de1f0d7b0d1dfb20ea4be81100c,2→1:$92.0; 4→2:$99.0,✅ Valid



⚠️ Tampering with a transaction in block 1...

📊 Blockchain Overview (after tampering):


,Block ID,Prev Hash,Curr Hash,Transactions,Status
0,1,0,bf0c29f6f797ad22514562b34cf3503f4e1fadf431d378a4b7302ff774ff7ee5,1→2:$9999.0; 2→3:$75.0,❌ Tampered
1,2,bf0c29f6f797ad22514562b34cf3503f4e1fadf431d378a4b7302ff774ff7ee5,573f308ea1b53165a4820f83b661023b3a415e7e7a117dee2c134a0970c276de,1→4:$109.0; 4→2:$176.0; 4→2:$91.0,✅ Valid
2,3,573f308ea1b53165a4820f83b661023b3a415e7e7a117dee2c134a0970c276de,1798ecfcbfeb56ae4ce2af6649382266ea705de1f0d7b0d1dfb20ea4be81100c,2→1:$92.0; 4→2:$99.0,✅ Valid
